# Testing all the models inside the "Models" folder

In [ ]:
import glob
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
from model import StocksPredictionModel

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

# Loading the model by name (with kwargs)

In [ ]:
def load_with_kwargs(model_name):
    kwargs, state_dict = torch.load(f"./Models/{model_name}")
    model = StocksPredictionModel(**kwargs)
    model.load_state_dict(state_dict)
    return model

# Predicting stock price

In [ ]:
def predict_stock_price(df, model):
    model = model.to(device)
    
    df.drop(["Open", "High", "Low"], axis = 1, inplace = True)
    
    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    df.dropna(inplace = True)
    close_price = df["Close"]
    
    preds_test = df.drop("Close", axis = 1)
    days_to_look = preds_test.tail(60)
    # 52 weeks since it's commonly used in financial analysis
    close_price = close_price.tail(364)

    days_to_look = days_to_look.to_numpy()
    close_price = close_price.to_numpy()
    
    test_scaler = StandardScaler()
    mm_scaler = MinMaxScaler()
    X_final = torch.Tensor(test_scaler.fit_transform(days_to_look.reshape(-1, days_to_look.shape[-1])).reshape(days_to_look.shape))
    
   # y_final = torch.Tensor(mm_scaler.fit_transform(close_price.reshape(-1, close_price.shape[-1])).reshape(close_price.shape))
    y_final = torch.Tensor(mm_scaler.fit_transform(close_price.reshape(1, -1))).reshape(close_price.shape)
    model.eval()
    with torch.no_grad():
        pred = model(X_final)
    
    preds = pred.detach().cpu()
    print(preds)
    
    preds = mm_scaler.inverse_transform(preds)
    return preds

# Testing the models on individual stock

In [ ]:
def test_all_models(root_directory, file_for_testing = "../Data/Stocks Data/Technology/Apple Inc. Common Stock.csv"):
    models = glob.glob(f"{root_directory}/**/*.pth", recursive=True)
    for model in models:
        print(os.path.basename(model))
        model_to_test = load_with_kwargs(model)
        predict_stock_price(file_for_testing, model_to_test)

In [ ]:
test_all_models("../Models")